In [1]:
from transformers import AutoTokenizer
import json
from tqdm import tqdm
import os

in_path = "../data/wiki_content.jsonl"
out_path = "../data/datastore_chunks.jsonl"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

In [2]:
tok = AutoTokenizer.from_pretrained("gpt2", use_fast=True)

chunk_size = 256
stride = 128 
written = 0

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
def chunk_text(text, chunk_size=256, stride=128):
    ids = tok.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(ids):
        end = min(start + chunk_size, len(ids))
        chunk = tok.decode(ids[start:end])
        chunks.append(chunk)
        if end == len(ids):
            break
        start += stride
    return chunks

In [4]:
with open(out_path, "w", encoding="utf-8") as fout:
    for i, line in enumerate(tqdm(open(in_path, "r", encoding="utf-8"), desc="Chunking")):
        obj = json.loads(line)
        title, text = obj["title"], obj["text"]
        for j, chunk in enumerate(chunk_text(text, chunk_size, stride)):
            fout.write(json.dumps({"doc_id": i, "title": title, "chunk_id": j, "chunk": chunk}, ensure_ascii=False) + "\n")
            written += 1

print(f"Wrote {written} chunks to {out_path}")

Chunking: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1402 > 1024). Running this sequence through the model will result in indexing errors
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Chunking: 1165it [00:35, 32.98it/s] 

Wrote 17575 chunks to ../data/datastore_chunks.jsonl
